In [ ]:
!pip install pydicom==2.0.0
!pip install pymedphys==0.33.0
!pip install scikit-image==0.17.2

In [ ]:
from urllib import request

import numpy as np
import matplotlib.pyplot as plt
import skimage.draw

import pydicom
import pymedphys

In [ ]:
dose_url = 'https://zenodo.org/record/4042842/files/RD.2.16.840.1.114337.1.1.1600065398.2_Anonymised.dcm?download=1'
structure_url = 'https://zenodo.org/record/4042842/files/RS.1.2.840.10008.5.1.4.1.1.481.3.1600145017_Anonymised.dcm?download=1'

dose_filepath = 'dose.dcm'
structure_filepath = 'structure.dcm'

In [ ]:
request.urlretrieve(dose_url, dose_filepath)
request.urlretrieve(structure_url, structure_filepath)

In [ ]:
dose_dataset = pydicom.read_file(dose_filepath, force=True)
structure_dataset = pydicom.read_file(structure_filepath, force=True)

In [ ]:
structure_names = [
    item.ROIName for item in structure_dataset.StructureSetROISequence
]

structure_names

In [ ]:
structure_names_to_pull = ['11', '12', '13', '14', '15', '16', '17', '18']

In [ ]:
(dose_z, dose_y, dose_x), dose = pymedphys.dicom.zyx_and_dose_from_dataset(dose_dataset)

In [ ]:
def pull_coords_from_contour_sequence(contour_sequence):
    contours_by_slice_raw = [item.ContourData for item in contour_sequence]

    x = [np.array(item[0::3]) for item in contours_by_slice_raw]
    y = [np.array(item[1::3]) for item in contours_by_slice_raw]
    z = [np.array(item[2::3]) for item in contours_by_slice_raw]

    return x, y, z


def get_roi_contour_sequence_by_name(structure_name, dcm_struct):
    ROI_name_to_number_map = {
        structure_set.ROIName: structure_set.ROINumber
        for structure_set in dcm_struct.StructureSetROISequence
    }

    ROI_number_to_contour_map = {
        contour.ReferencedROINumber: contour
        for contour in dcm_struct.ROIContourSequence
    }

    try:
        ROI_number = ROI_name_to_number_map[structure_name]
    except KeyError:
        raise ValueError("Structure name not found (case sensitive)")

    roi_contour_sequence = ROI_number_to_contour_map[ROI_number]

    return roi_contour_sequence


def pull_structure(structure_name, dcm_struct):
    roi_contour_sequence = get_roi_contour_sequence_by_name(structure_name, dcm_struct)
    contour_sequence = roi_contour_sequence.ContourSequence

    x, y, z = pull_coords_from_contour_sequence(contour_sequence)

    return x, y, z

In [ ]:
stucture_x, structure_y, structure_z = pull_structure('11', structure_dataset)
z_value = np.unique(structure_z[0])
z_value

In [ ]:
index = np.where(dose_z == z_value)[0][0]
index

In [ ]:
plt.contourf(dose_x, dose_y, dose[index,:,:], 100)

plt.plot(stucture_x[0], structure_y[0])

plt.ylim([-50, -350])
plt.axis('equal')

In [ ]:
position = dose_dataset.ImagePositionPatient
spacing = dose_dataset.PixelSpacing
orientation = dose_dataset.ImageOrientationPatient

dx, dy = spacing
Cx, Cy, *_ = position
Ox, Oy = orientation[0], orientation[4]

r = (y - Cy) / dy * Oy
c = (x - Cx) / dx * Ox



In [ ]:
dose_dataset.pixel_array.shape

In [ ]:
skimage.draw.polygon2mask(dose_dataset.pixel_array.shape, np.array(zip(r, c)))